Случайные леса
===============

Используемые дополнительно ссылки.

https://habr.com/ru/post/349078/<br>
https://habr.com/ru/company/ods/blog/324402/#2-sluchaynyy-les <br>


In [1]:
# Подключаю необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

В лабораторной работе 1 я не стала ничего делать с данным (например, балансировать распределение по классам, искать корреляцию или исключать какие-либо критерии), для оценки качества модели использовала roc_auc_ovo_weighted - на обучающей 0.7, на тестовой - 0.59. В этот раз хочу пойти другим путем.

In [2]:
#  Импортирую данные
df = pd.read_csv('winequality-red.csv', sep=';')

In [3]:
#  проверяю как все загрузилось
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
# смотрю распределение по классам
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
#Выборка не сбалансирована, попробуем перекинуть класс 3 к 4, класс 8 к 7.
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7)

In [6]:
# смотрю новое распределение по классам
df['quality'].value_counts()

5    681
6    638
7    217
4     63
Name: quality, dtype: int64

In [7]:
#  разделяю предикоторы и отклики
y = df['quality']
X = df.drop(columns=['quality'])

In [8]:
# Проверяю себя
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [9]:
# Проверяю себя
y.head()

0    5
1    5
2    5
3    6
4    5
Name: quality, dtype: int64

In [10]:
#  Разделяю на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1234)  

##  Подбор параметров вручную

In [11]:
# Для ручного подбора параметров пользуюсь статьей (указана в источниках)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=1000,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # Максимальная глубина деревьев
                               max_depth=10,
                               # сколько переменных использовать при построении дерева
                               max_features='auto', 
                               # максимальное число конечных узлов. Сейчас - без ограничений
                               max_leaf_nodes=None,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.0, 
                               # Устарело
                               min_impurity_split=None,
                               # Дерево обучается по части наблюдений
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=10, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=2,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight='balanced')

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.8s finished


RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=10, n_estimators=1000, oob_score=True,
                       random_state=42, verbose=1)

In [12]:
from sklearn import metrics
# Строим предсказание модели
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
# и оцениваем ее (больше всего интересует f1-score)
print(metrics.classification_report(y_pred_test, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


              precision    recall  f1-score   support

           4       0.30      0.19      0.23        16
           5       0.75      0.75      0.75       137
           6       0.47      0.65      0.55        95
           7       0.85      0.49      0.62        72

    accuracy                           0.63       320
   macro avg       0.59      0.52      0.54       320
weighted avg       0.67      0.63      0.63       320



[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


f1-score = 0.63, для сравнения на одиночном дереве было 0.58

In [14]:
from sklearn.metrics import balanced_accuracy_score
print("Balanced accuracy on train set is ", balanced_accuracy_score(y_train, y_pred_train)*100)
print("Balanced accuracy set is ", balanced_accuracy_score(y_test, y_pred_test)*100)

Balanced accuracy on train set is  84.07939741410311
Balanced accuracy set is  59.33294477319151


In [15]:
# Меня качество устраивает, придержусь принципа Парето и не буду проводить GridSearch 
# еще посмотрим что и как влияло на построение
pd.DataFrame({'feature': X_train.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
10,alcohol,0.182579
1,volatile acidity,0.161564
9,sulphates,0.158409
6,total sulfur dioxide,0.098212
2,citric acid,0.076757
7,density,0.060994
8,pH,0.058304
4,chlorides,0.056371
0,fixed acidity,0.049266
5,free sulfur dioxide,0.048991


Информативность растет очень плавно от переменной к переменной, если буду в будущем что-то отбрасывать, то по одной за раз.

In [16]:
#посмотрим матрицу путаницы для тестовой
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,4,5,6,7
4,3,4,2,1
5,8,103,26,1
6,5,29,62,35
7,0,1,5,35


4 почти не определяется, 7 определяется одинаково часто и как 7, и как 6. Я это могу интерпретировать как то, что классификатор видит только 3 вида вина: 1) качество ниже среднего 2) качество среднее 3) качество выше среднего <br>
Меня как потребителя, а не как сомелье, такое распределение устраивает

In [17]:
#посмотрим еще матрицу путаницы для тренировочной
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,4,5,6,7
4,53,0,0,0
5,37,410,76,20
6,16,75,340,76
7,1,4,6,165


In [18]:
# Строим предсказание модели для тестовой выборки
y_pred_test2 = model.predict_proba(X_test)
print(y_pred_test2)

[[0.43870007 0.3666188  0.12813175 0.06654939]
 [0.11863178 0.45218815 0.36796045 0.06121963]
 [0.07099655 0.23805238 0.38940184 0.30154923]
 ...
 [0.02610966 0.11061716 0.37618487 0.48708831]
 [0.23423365 0.14070492 0.35576013 0.2693013 ]
 [0.15072358 0.58523505 0.23560868 0.02843269]]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


In [19]:
#посмотрим размер матрицы с вероятностями отнесения к классу
y_pred_test2.shape

(320, 4)

In [20]:
#считаем долю
x=0
for i in range(320):
    for j in range(4):
        if (y_pred_test2[i][j])>0.8:
            x+=1
print((x/320*100), "% наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8")

3.75 % наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8


## Калибровка

In [21]:
from sklearn.calibration import CalibratedClassifierCV

In [22]:
#метод калибровки соответствует логистической регрессионной модели
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')

In [23]:
model_sigmoid.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


CalibratedClassifierCV(base_estimator=RandomForestClassifier(class_weight='balanced',
                                                             max_depth=10,
                                                             min_samples_leaf=10,
                                                             n_estimators=1000,
                                                             oob_score=True,
                                                             random_state=42,
                                                             verbose=1),
                       cv=2)

In [302]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


              precision    recall  f1-score   support

           4       0.10      1.00      0.18         1
           5       0.80      0.74      0.77       151
           6       0.56      0.61      0.58       122
           7       0.54      0.48      0.51        46

    accuracy                           0.65       320
   macro avg       0.50      0.70      0.51       320
weighted avg       0.67      0.65      0.66       320



[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.1s finished


f1-score с 0.63 поднялся до 0.65